In [6]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from ultralytics import RTDETR

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="2kT053S4dqKLHoB6yTpk")
    project = rf.workspace("bitter-gourd-augemented-dataset").project("eyes-jp15h")
    version = project.version(3)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = RTDETR("rtdetr-l")   # Adjust model if needed




Dataset already exists. Using the existing dataset.


In [7]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "valid", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(5):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/test/labels

Number of images per class:
Class 0: 6178 images
Class 1: 601 images
Class 2: 1293 images
Class 3: 196 images
Class 4: 162 images


In [8]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="Eyepac Org RTdetr",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=32,  # Adjust batch size for better GPU utilization
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=8, project=None, name=Eyepac Org RTdetr3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, sp

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/train/labels.cache... 26696 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26696/26696 [00:00<?, ?it/s]
val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels.cache... 5971 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5971/5971 [00:00<?, ?it/s]


Plotting labels to runs/detect/Eyepac Org RTdetr3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 32 dataloader workers
Logging results to runs/detect/Eyepac Org RTdetr3
Starting training for 100 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/835 [00:00<?, ?it/s]/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Cont

                   all       5971       5971      0.783      0.202      0.178      0.177

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

                   all       5971       5971      0.695      0.601      0.527      0.526

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/835 [00:00<?, ?it/s]/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Cont

                   all       5971       5971      0.688      0.599      0.524      0.524

100 epochs completed in 20.546 hours.
Optimizer stripped from runs/detect/Eyepac Org RTdetr3/weights/last.pt, 66.2MB
Optimizer stripped from runs/detect/Eyepac Org RTdetr3/weights/best.pt, 66.2MB

Validating runs/detect/Eyepac Org RTdetr3/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
rt-detr-l summary: 302 layers, 31,994,015 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 374/374 [01:31<00:00,  4.07it/s]


                   all       5971       5971      0.694      0.601      0.527      0.526
                     0       4379       4379      0.916      0.967      0.938      0.938
                     1        412        412       0.44      0.337      0.197      0.197
                     2        923        923      0.733      0.662      0.573      0.572
                     3        144        144      0.505      0.403      0.293      0.293
                     4        113        113      0.878      0.637      0.632      0.632
Speed: 0.2ms preprocess, 8.5ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/Eyepac Org RTdetr3


In [9]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
rt-detr-l summary: 302 layers, 31,994,015 parameters, 0 gradients, 103.5 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels.cache... 5971 images, 0 backgrounds, 0 corrupt: 100%|██████████| 597
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [02:03<00:00,  1.52it/s]


                   all       5971       5971      0.694      0.601      0.527      0.526
                     0       4379       4379      0.916      0.967      0.938      0.938
                     1        412        412       0.44      0.337      0.197      0.197
                     2        923        923      0.734      0.662      0.572      0.572
                     3        144        144      0.501      0.403      0.293      0.293
                     4        113        113      0.878      0.637      0.632      0.632
Speed: 0.2ms preprocess, 14.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/Eyepac Org RTdetr32
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd4ca3f9c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'R

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)